In [5]:
import pandas as pd
import datetime
import re
import numpy as np
import numpy.polynomial.polynomial as poly
import sys, os
from math import pi
import re

from scipy.optimize import curve_fit

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from numpy import inf
np.random.seed(123)
from sklearn.metrics import r2_score

from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import BuPu9, GnBu9, Category20c, RdYlBu, OrRd, RdBu
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, BoxSelectTool, Range1d, Rect, LabelSet, BooleanFilter, BoxAnnotation, DatetimeTickFormatter, CDSView, GroupFilter, NumeralTickFormatter, Label
from bokeh.layouts import row
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.transform import linear_cmap

from bokeh.plotting import figure
from bokeh.transform import cumsum

output_notebook()

#Plotting options
opts = dict(width=1200, height=600, toolbar_location="above",
      tools='tap, box_zoom, pan, undo, crosshair, reset, wheel_zoom, box_select, save')

Loading BokehJS ...

In [6]:
#Return list of cases per state and county
def populateCases(state, county, population=None):
    state = str(state)
    county = str(county)
    listToReturn = []
    
    for date in dfDates:
        tempValue = df[ (df['Province_State'] == state) & (df['Admin2'] == county) ][date].tolist()[0]
        
        if population == None:
            listToReturn.append(int(tempValue))
        else:
            listToReturn.append(float(tempValue/population))
            
    #print(state, county, listToReturn)
    return listToReturn


#Return Population per state and county
def getPopulation(state, county):
    #New York is split into two counties. Sum them. Populate cases is for NYC
    if (state == 'New York') & (county == 'New York'):
        popSeries = dfPopulate['POPESTIMATE2019'][ (dfPopulate['STNAME'] == state) & (dfPopulate['CTYNAME'] == 'Bronx County') ]
        #print(popSeries)
        pop = popSeries.tolist()[0]
        popSeries = dfPopulate['POPESTIMATE2019'][ (dfPopulate['STNAME'] == state) & (dfPopulate['CTYNAME'] == 'Kings County') ]
        #print(popSeries)
        pop = pop + popSeries.tolist()[0]
        pop = 8600000000
    else:
        county = county + ' County'
        popSeries = dfPopulate['POPESTIMATE2019'][ (dfPopulate['STNAME'] == state) & (dfPopulate['CTYNAME'] == county) ]
        pop = popSeries.tolist()[0]
        #print(pop)
    return pop



In [7]:
#Covid Data
gitFile = 'COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

#Population Data
popFile = 'co-est2019-alldata.csv'

#States and Counties to get data for
dataToGrab = [
              ['Florida', 'Palm Beach'], ['Texas', 'Dallas'], 
              #['New York', 'New York'], 
              #['California', 'Los Angeles'],
              ['Georgia', 'Fulton'], ['Georgia', 'DeKalb'],
              #['Florida', 'Miami-Dade'], 
              #['Massachusetts', 'Suffolk'],
              #['New Jersey', 'Essex'],
              ['Tennessee', 'Davidson'], 
              ['Tennessee', 'Hamilton'], ['Tennessee', 'Knox'], ['Kentucky', 'Fayette'],
             ]

#Divide by population - T/F
divideByPopulation = False

#Read CSVs
df = pd.read_csv(gitFile)
dfPopulate = pd.read_csv(popFile, engine='python')
 
#Append population
for pop in dataToGrab:
    population = getPopulation(pop[0], pop[1])
    #Add population to list
    pop.append(population)
    
    
#Dates are embedded into the columns - Ignore first 11 columns
dfDates = list(df.columns)[11:] 

#Create empty list for storing multiple counties
countyStateCases = {}
for itm in dataToGrab:
    #Get cases per location
    if divideByPopulation:
        tempData = populateCases(itm[0], itm[1], itm[2])
    else:
        tempData = populateCases(itm[0], itm[1])
    #Combine county+_+state for key
    key = str(itm[0])+ '_' +str(itm[1])
    countyStateCases[ key ] = tempData

#Get dates as datetime
dataDates = [datetime.datetime.strptime(x, '%m/%d/%y') for x in dfDates]

#Convert to DF
dfToPlot = pd.DataFrame.from_dict( countyStateCases )
dfToPlot = dfToPlot.diff()

#Append dates to dict
dfToPlot[ 'dates' ] = dataDates

#dfToPlot

In [8]:
#Plot data
pltA = figure(**opts, x_axis_type="datetime")

#Column names are the counties to plot
columnCounties = dfToPlot.columns.tolist()
columnCounties.remove('dates');

#Gets dates
columnCountiesDates = dfToPlot['dates']

#Add data from dfToPlot
for idx, location in enumerate(columnCounties, start=0):
    #Parse Name
    legendName = location.replace('_', ', ')
    #Get color
    lineColor = str(RdBu[len(columnCounties)][idx])
    
    #Plot
    pltA.vbar(x=dfToPlot['dates'], top=dfToPlot[location], width=46000000, legend_label=legendName, color=lineColor, alpha=0.5)

#Update location of the legend
pltA.legend.location = 'top_left'
pltA.legend.click_policy="hide"

#Update Title

pltA.title.text = 'Cases in Select US Counties'
pltA.yaxis.axis_label = 'Number of New Daily Cases'

#Add Hatch Pattern
pltA.xgrid.band_hatch_pattern = "\\"
pltA.xgrid.band_hatch_alpha = 0.4
pltA.xgrid.band_hatch_color = "lightgrey"
pltA.xgrid.band_hatch_weight = 0.4
pltA.xgrid.band_hatch_scale = 20

#Format to always show Month/Day
pltA.xaxis.formatter=DatetimeTickFormatter(days="%m/%d",
    months="%m/%d",
    hours="%m/%d",
    minutes="%m/%d")

#Add more ticks!
pltA.xaxis.ticker.desired_num_ticks = int(len(dfToPlot)/4)

pltA.background_fill_color = "#adadad"

show(pltA)